In [1]:
%%capture
from pathlib import Path
from ccdproc import ImageFileCollection
from photozpy import Telescope, HeaderCorrection, Telescope, SwiftDownload, mImageFileCollection, SwiftCombine, PhotozRegions
import pandas as pd
import copy
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import shutil
import os
from astropy.io import fits
os.environ["HEADASNOQUERY"]=""
os.environ['HEADASPROMPT'] = '/dev/null'

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

## Define the root directory to download and analyze data

In [2]:
# set to the directory as you desire. It's the root directory of this notebook in default
root_dir = Path("./data")
root_dir.mkdir(exist_ok=True)

## Download data

In [7]:
# create the swift download instance
swift_download = SwiftDownload(download_dir = root_dir)

# parse the obsquery infor from a csv file
swift_download.set_obsquery_info_from_csv("group_1_source_catalog.csv")

# download data
swift_download.download_swift_data(organize = True)

00038162004 on 2021-01-27 09:03:02 is being downloaded, the uvot mode is 0x30ed


00038162004 has been downloaded/examined, skipping ...
00014040001 on 2021-02-05 18:02:01 is being downloaded, the uvot mode is 0x30ed


00014040001 has been downloaded/examined, skipping ...
00014040001 has been downloaded/examined, skipping ...
00014051002 on 2021-02-10 11:02:02 is being downloaded, the uvot mode is 0x30ed


00014051001 on 2021-02-07 16:01:02 is being downloaded, the uvot mode is 0x30ed


00014051001 has been downloaded/examined, skipping ...
00014051001 has been downloaded/examined, skipping ...
00014073001 on 2021-02-17 10:12:02 is being downloaded, the uvot mode is 0x30ed


...

00083796004 on 2021-03-16 06:16:01 is being downloaded, the uvot mode is 0x30ed


00083796004 has been downloaded/examined, skipping ...
00083796003 on 2021-03-15 13:44:02 is being downloaded, the uvot mode is 0x30ed


00039147007 on 2021-03-30 22:16:02 is being downloaded, the uvot mode is 0x30ed


00039147007 has been downloaded/examined, skipping ...
00014215002 on 2021-04-13 16:08:02 is being downloaded, the uvot mode is 0x30ed


00014215002 has been downloaded/examined, skipping ...


## Sum extensions within and across observations

In [3]:
# Get the directory for each source (only when the source has obervations)

paths = [path for path in root_dir.iterdir() if any(path.iterdir())]

for path in paths:
    print(path)

# Hidden directories will be included if exists, make sure you remove the directory that you don't want.
#paths.remove(paths[1])
#print(paths)

data/Ton_1015
data/1RXS_J085930.5+621737
data/B3_0850+443
data/SDSS_J085135.93+552834.5
data/GB6_J0850+4855
data/TXS_0846+051
data/SDSS_J081649.78+205106.4
data/NVSS_J084411+531250
data/NVSS_J132758+252750


In [4]:
# Define a multi image collection object
multi_collection = mImageFileCollection(image_dir = paths)

# Define the swift telescope
swift_telescope = Telescope(telescope = "Swift", mode = "0X30ED", ccd = "UVOT")

In [5]:
# Sum the extensions
swift_combine = SwiftCombine(image_collection = multi_collection, telescope = swift_telescope)

In [6]:
swift_combine.sum_all_files()

uvotimsum infile=data/Ton_1015/sw00038162004ubb_sk.fits outfile=data/Ton_1015/B.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00038162004ubb_sk.fits has been summed to B.fits

uvotimsum infile=data/Ton_1015/sw00038162004um2_sk.fits outfile=data/Ton_1015/UVM2.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00038162004um2_sk.fits has been summed to UVM2.fits

uvotimsum infile=data/Ton_1015/sw00038162004uuu_sk.fits outfile=data/Ton_1015/U.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00038162004uuu_sk.fits has been summed to U.fits

uvotimsum infile=data/Ton_1015/sw00038162004uvv_sk.fits outfile=data/Ton_1015/V.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00038162004uvv_sk.fits has been summed to V.fits

uvotimsum infile=data/Ton_1015/sw00038162004uw1_sk.fits outfile=data/Ton_1015/UVW1.fits | tee -a uvotimsum_log.txt >/dev/null 2>&1
Extensions in sw00038162004uw1_sk.fits has been summed to UVW1.fits

uvotimsum infile=d

# Generate region files for sources

In [5]:
# Get the directory for each source (only when the source has obervations)

paths = [path for path in root_dir.iterdir() if any(path.iterdir())]

for path in paths:
    print(path)

# Define a multi image collection object
multi_collection = mImageFileCollection(image_dir = paths, glob_exclude = "*_*")

# Define the swift telescope
swift_telescope = Telescope(telescope = "Swift", mode = "0X30ED", ccd = "UVOT")

data/Ton_1015
data/1RXS_J085930.5+621737
data/B3_0850+443
data/SDSS_J085135.93+552834.5
data/GB6_J0850+4855
data/TXS_0846+051
data/SDSS_J081649.78+205106.4
data/NVSS_J084411+531250
data/NVSS_J132758+252750


In [6]:
source_regions = PhotozRegions(image_collection = multi_collection, 
                               telescope = swift_telescope, 
                               source_catalog_path = "group_1_source_catalog.csv")

In [7]:
#%%capture
source_regions.get_source_regions(box_size = 15, save_image = True, bkg_region_template = "bkg.reg")

Generating source regions for Ton 1015
The centroid pixel for B is ([552.25978478],[548.19604971]).
The centroid pixel for U is ([550.94162213],[551.10112742]).
The centroid pixel for UVM2 is ([554.55207322],[549.28539744]).
The centroid pixel for UVW1 is ([555.83274399],[554.79383343]).
The centroid pixel for UVW2 is ([555.64230436],[550.7820387]).
The centroid pixel for V is ([553.12655677],[549.74464642]).
--------------------------------------------------------------------
Generating source regions for 1RXS J085930.5+621737
The centroid pixel for B is ([761.07142971],[819.32221662]).
The centroid pixel for U is ([761.0168139],[822.26190951]).
The centroid pixel for UVM2 is ([760.87615259],[817.99788007]).
The centroid pixel for UVW1 is ([756.44982595],[815.01075137]).
The centroid pixel for UVW2 is ([762.96553212],[821.7064713]).
The centroid pixel for V is ([760.0353215],[818.5412688]).
--------------------------------------------------------------------
Generating source regions 

In [20]:
from astropy.nddata import CCDData
ccddata = CCDData.read("data/Ton_1015/B.fits", output_verify = "silentfix")

INFO:astropy:first HDU with data is extension 1.


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]


In [22]:
from astropy.io import fits

hdul = fits.open("data/Ton_1015/B.fits")

In [29]:
df = pd.read_csv("group_1_source_catalog.csv", sep = ",")
df[df["name"] == "Ton 1015"]["ra"]

0    137.654292
Name: ra, dtype: float64

In [21]:
df

,name,targetid,window_lower,window_upper,ra,dec
0,Ton 1015,38162,2021-01-24,2021-02-26,137.654292,33.490119
1,1RXS J085930.5+621737,14040,2021-02-02,2021-03-07,134.877958,62.291800
2,B3 0850+443,14051,2021-02-04,2021-03-09,133.541208,44.141750
3,SDSS J085135.93+552834.5,14073,2021-02-14,2021-03-19,132.899708,55.476231
4,GB6 J0850+4855,14083,2021-02-21,2021-03-26,132.501458,48.916300
5,GB6 J0848+6605,14092,2021-02-28,2021-04-02,132.227583,66.102639
6,TXS 0846+051,14106,2021-03-04,2021-04-06,132.385625,4.918850
7,SDSS J081649.78+205106.4,83796,2021-03-12,2021-04-14,124.207417,20.851789
8,NVSS J084411+531250,39147,2021-03-20,2021-04-22,131.048750,53.214050
9,NVSS J132758+252750,14215,2021-04-02,2021-05-05,201.994375,25.463939


In [31]:
df[df["name"] == "1RXS J085930.5+621737"]["ra"].to_numpy()[0]

134.87795833333

In [14]:
skycoord = SkyCoord(ra = df[df["name"] == "Ton 1015"]["ra"][0], dec = df[df["name"] == "Ton 1015"]["dec"][0], unit = "deg", frame = "icrs")

In [15]:
skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    (137.65429167, 33.49011944)>

In [6]:
a = "Ton_344"

In [7]:
a.replace("_", " ")

'Ton 344'

In [6]:
collection_path = multi_collection[0].location
source_name = collection_path.parts[-1].replace("_", " ")

In [8]:
collection_path.parts[-1]

'Ton_1015'